In [ ]:
import numpy as np
import tensorflow as tf
import glob as gb
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# get training data
img_trainpath = gb.glob("training//*.jpg")
train_x = []
train_y = []
for path in img_trainpath:
    pic = Image.open(path)
    pix = np.array(pic)
    train_x.append(pix)
    tmp = str(path)
    a = tmp.split('/')[1].split('_')[0]
    a = int(a)
    onehot_tmp = np.zeros(11)
    onehot_tmp[a] = 1
    onehot_tmp = np.array(onehot_tmp)
    train_y.append(onehot_tmp)

train_y = np.array(train_y)
train_x = np.array(train_x)

In [ ]:
print(train_x.shape)
print(img_trainpath[0])
print(train_y[0])
#tmp = gb.glob('training//*.jpg')
#print(tmp[0])

In [ ]:
# get test data
img_testpath = gb.glob("evaluation//*.jpg")
test_x = []
test_y = []
for path in img_testpath:
    pic = Image.open(path)
    pix = np.array(pic)
    #pix = np.array(pic.getdata()).reshape(pic.size[0], pic.size[1], 3)
    test_x.append(pix)
    tmp = str(path)
    a = tmp.split('/')[1].split('_')[0]
    a = int(a)
    onehot_tmp = np.zeros(11)
    onehot_tmp[a] = 1
    onehot_tmp = np.array(onehot_tmp)
    test_y.append(onehot_tmp)
test_y = np.array(test_y)
test_x = np.array(test_x)

In [ ]:
print(test_x.shape)
print(img_testpath[0])
print(test_y[0])

In [ ]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape,  mean=0, stddev=0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.truncated_normal(shape=shape,mean=0, stddev=0.05)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
# placeholder
xs = tf.placeholder(tf.float32, [None, 96,96,3])/255.   
ys = tf.placeholder(tf.float32, [None, 11])
keep_prob = tf.placeholder(tf.float32)
print(xs.shape)

In [ ]:
## conv1,max1 layer ##
W_conv1 = weight_variable([3,3,3,32]) # patch 3x3, in size 3, out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.selu(conv2d(xs, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)                      #### output size 48x48x32

## conv2,max2 layer ##
W_conv2 = weight_variable([3,3,32,64]) # patch 3x3, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.selu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)                      #### output size 24x24x64

## conv3,max3 layer ##
W_conv3 = weight_variable([3,3,64,128]) # patch 3x3, in size 64, out size 128
b_conv3 = bias_variable([128])
h_conv3 = tf.nn.selu(conv2d(h_pool2, W_conv3) + b_conv3)  
h_pool3 = max_pool_2x2(h_conv3)                      #### output size 12x12x128

## conv4,max4 layer ##
W_conv4 = weight_variable([3,3,128,256]) # patch 3x3, in size 128, out size 256
b_conv4 = bias_variable([256])
h_conv4 = tf.nn.selu(conv2d(h_pool3, W_conv4) + b_conv4)  
h_pool4 = max_pool_2x2(h_conv4)                      #### output size 6x6x256

## fc1 layer ##
W_fc1 = weight_variable([6*6*256, 1024])
b_fc1 = bias_variable([1024])
# [n_samples, 6, 6, 256] ->> [n_samples, 6*6*256]
h_pool4_flat = tf.reshape(h_pool4, [-1, 6*6*256])
h_fc1 = tf.nn.selu(tf.matmul(h_pool4_flat, W_fc1) + b_fc1)

## fc2 layer ##
W_fc2 = weight_variable([1024, 1024])
b_fc2 = bias_variable([1024])
h_fc2 = tf.nn.selu(tf.matmul(h_fc1, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

W_fc3 = weight_variable([1024, 11])
b_fc3 = bias_variable([11])
prediction = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

In [ ]:
# loss
value = tf.trainable_variables()
l2_loss = 0.
lamda = 1
for i in value:
    l2_loss += tf.norm(i)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=
                                                        prediction, labels=ys)) + lamda * l2_loss
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction) + (1-ys) * tf.log(1-prediction),
#                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# training
total_train = train_x.shape[0]
total_test = test_x.shape[0]
batch_size = 64
test_batch_size = 64
iteration = 50
train_accuracy = []
test_accuracy = []
loss_ = []

In [ ]:
# important step initial all the variables
sess = tf.Session()
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
for i in range(iteration):
    #training
    for j in range(total_train % batch_size - 1):
        tmp_x = train_x[j*batch_size:j*batch_size+batch_size,:,:,:]
        tmp_y = train_y[j*batch_size:j*batch_size+batch_size,:]
        sess.run(train_step, feed_dict={xs: tmp_x, ys: tmp_y, keep_prob: 0.6})
    loss_tmp = []
    train_acc = []
    test_acc = []
    # calculate training_accuracy and loss
    for m in range(total_train % test_batch_size - 1):
        tmp_x = train_x[m*test_batch_size:m*test_batch_size+test_batch_size,:,:,:]
        tmp_y = train_y[m*test_batch_size:m*test_batch_size+test_batch_size,:]
        loss_tmp.append(sess.run(cross_entropy,feed_dict={xs: tmp_x, 
                                                     ys: tmp_y, keep_prob: 1}))
        train_acc.append(compute_accuracy(tmp_x, tmp_y))
    #calculate test_accuracy    
    for n in range(total_test % test_batch_size - 1):
        tmp_x = test_x[n*test_batch_size:n*test_batch_size+test_batch_size,:,:,:]
        tmp_y = test_y[n*test_batch_size:n*test_batch_size+test_batch_size,:]
        test_acc.append(compute_accuracy(tmp_x, tmp_y))
    train_tmp_acc = np.sum(train_acc)/(m + 1)
    test_tmp_acc = np.sum(test_acc)/(n + 1)
    loss_tmp_ = np.sum(loss_tmp)/(m + 1)
    loss_.append(loss_tmp_)
    train_accuracy.append(train_tmp_acc)
    test_accuracy.append(test_tmp_acc)
    print('iteration:',i,'train_accuracy:',train_tmp_acc,'test_accuracy:',test_tmp_acc)
    print('loss:',loss_tmp_)

In [ ]:
t = np.arange(50)
plt.plot(t,loss_)
plt.title('Learing Curve with L2 norm')
plt.xlabel('Iteration')
plt.ylabel('Cross Entropy')
plt.show()

In [ ]:
p1 =plt.plot(t,train_accuracy,'r', label='training_accuracy')
p2 = plt.plot(t,test_accuracy,'b', label='test_accuracy')
plt.title('training and test accuray with L2 norm')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.legend(loc='lower right') 
plt.show()

In [ ]:
a = W_conv1.eval(session=sess)
binwidth = 0.001
a = a.flatten()
a.shape
plt.hist(a,bins = 50)  # arguments are passed to np.histogram
plt.title("Histogram of conv1")
plt.show()
a = W_conv2.eval(session=sess)
a = a.flatten()
a.shape
plt.hist(a,bins = 50)  # arguments are passed to np.histogram
plt.title("Histogram of conv2")
plt.show()
a = W_conv3.eval(session=sess)
a = a.flatten()
a.shape
plt.hist(a,bins = 50)  # arguments are passed to np.histogram
plt.title("Histogram of conv3")
plt.show()
a = W_conv4.eval(session=sess)
a = a.flatten()
a.shape
plt.hist(a,bins = 50)  # arguments are passed to np.histogram
plt.title("Histogram of conv4")
plt.show()
a = W_fc1.eval(session=sess)
a = a.flatten()
a.shape
plt.hist(a,bins = 50)  # arguments are passed to np.histogram
plt.hist(a,bins = 50)
plt.title("Histogram of fc1")
plt.show()
a = W_fc2.eval(session=sess)
a = a.flatten()
a.shape
plt.hist(a,bins = 50)  # arguments are passed to np.histogram
plt.title("Histogram of fc2")
plt.show()
a = W_fc3.eval(session=sess)
a = a.flatten()
a.shape
plt.hist(a,bins = 50)  # arguments are passed to np.histogram
plt.title("Histogram of fc3")
plt.show()

In [ ]:
import matplotlib.image as mpimg
a1 = sess.run(h_pool1,feed_dict={xs: train_x[3500:3501,:,:,:], keep_prob: 1})

img = train_x[500,:,:,:]
a1 = np.array(a1).reshape([48,48,32])
print(a1.shape)
print(train_y[3500,:])
#img = a1[:,:,16]
imgplot = plt.imshow(img)
plt.show()

In [ ]:
tf.norm(W_fc1)